In [ ]:
# import copy for copying lists
import copy

# import regular expression package for string manipulation
import re

# import SymPy package for symoblic computation
from sympy import symbols, Add, Mul

In [ ]:
%run quantum_one_time_pad.ipynb
%run che_initialization.ipynb
%run epr_encryption.ipynb

In [ ]:
def tcl_layers(circuit):
    """
    Seperate T + Clifford gates of each layer of a quantum circuit.
    
    Args:
        - circuit: a quantum circuit as a list of strings. each member of the list
        represents a layer in the quantum circuit.
        
    Returns:
        - layered_circuit: a nested list where each nested list seperates individual gates. 
    """
    qgates = ["CNOT", "Z", "I", "X", "P", "H", "T"]
    layered_circuit = []
    
    length_current_layer = 0 
    for layer in range(len(circuit)): 
        length_current_layer = len(circuit[layer])
        counter = 0
        current_gates = []
        for gates in range(length_current_layer):
            if counter >= length_current_layer:
                break
            if circuit[layer][counter] != "C":
                current_gates.append(circuit[layer][counter])
                counter += 1
            else:
                current_gates.append("CNOT")
                counter += 4
        layered_circuit.append(current_gates)
        
    return layered_circuit

In [ ]:
def clifford_key_updates(cg, a, b):
    """
    Perform key updates homomorphically on ciphertexts depending on based on a Clifford gate.
    
    Args: 
        - cg: Clifford gate as a string ("I","X","Z","H","P","CNOT")
        - a: ciphertext
        - b: ciphertext
        
    Returns: 
        tuple 
            If the Clifford gate is a single qubit gate:
                - a tuple of updated ciphertexts
            If the Clifford gate is the CNOT gate:
                - a tuple of lists, each containing the updated ciphertexts for the current wire and 
                its neighboring wire for a and b, respectively. 
    """
    if cg == 'X' or cg == 'Z' or cg == 'I':
        return (a,b)
    
    if cg == 'H':
        return(b,a)
    
    if cg == 'P': 
        return(a, evaluator.add(a, b))

    
    if cg == 'CNOT':
        return ([a[0], evaluator.add(a[0],a[1])] ,[evaluator.add(b[0],b[1]), b[1]])

In [ ]:
def cl_eval(circuits, enc_psi, a_tilde, b_tilde):
    """
    Perform clifford circuit on the encrypted pure state homomorphically
    and update the classical ciphertexts using classical homomorphic encryption scheme.
    For more information refer to the Clifford scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args: 
        - circuits: description of the Clifford circuit as a list of strings
        - enc_psi: a vector representation of the encrypted pure state |psi>_{enc}
        - a_tilde: a list of classical ciphertexts which contain the information about the X-Pauli gate exponents 
        - b_tilde: a list of classical ciphertexts which contain the information about the Z-Pauli gate exponents 
        
    Returns:
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - a_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the X-Pauli gate exponents 
        - b_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the Z-Pauli gate exponents 
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 
    """
    # making copies of ciphertexts
    a_tilde_copy, b_tilde_copy = a_tilde, b_tilde
    a_tilde_update, b_tilde_update = a_tilde, b_tilde
    
    # density matrix representation of the encrypted state
    rho_hom_eval = pure_to_density(enc_psi)
    
    # dimension of the quantum state
    n = len(a_tilde)
    
    for i in tcl_layers(circuits):
        counter = 0 
        
        # current circuit iteratively build up the quantum gate in each layer of the ciruit 
        current_circuit = np.array([1])
        for j in range(len(i)):
            if i[j] == 'I':
                current_circuit = np.kron(current_circuit, I)
                a_tilde_update[counter], b_tilde_update[counter] = clifford_key_updates(
                    i[j], a_tilde_copy[counter], b_tilde_copy[counter]
                )
                counter = counter + 1
                
            if i[j] == 'X':
                current_circuit = np.kron(current_circuit, X)
                a_tilde_update[counter], b_tilde_update[counter] = clifford_key_updates(
                    i[j], a_tilde_copy[counter], b_tilde_copy[counter]
                )
                counter = counter + 1

            if i[j] == 'Z':
                current_circuit = np.kron(current_circuit, Z)
                a_tilde_update[counter], b_tilde_update[counter] = clifford_key_updates(
                    i[j], a_tilde_copy[counter], b_tilde_copy[counter]
                )
                counter = counter + 1

            if i[j] == 'H':
                current_circuit = np.kron(current_circuit, H)
                a_tilde_update[counter], b_tilde_update[counter]= clifford_key_updates(
                    i[j], a_tilde_copy[counter], b_tilde_copy[counter]
                )
                counter = counter + 1

            if i[j] == 'P':
                current_circuit = np.kron(current_circuit, P)
                a_tilde_update[counter], b_tilde_update[counter] = clifford_key_updates(
                    i[j], a_tilde_copy[counter], b_tilde_copy[counter]
                )
                counter = counter + 1

            if i[j] == 'CNOT':
                current_circuit = np.kron(current_circuit, CNOT)
                a_tilde_update[counter:counter+2], b_tilde_update[counter:counter+2] = clifford_key_updates(
                    i[j], a_tilde_copy[counter:counter+2], b_tilde_copy[counter:counter+2]
                )
                counter = counter + 2
        rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
    
    circuit_dictionary = wire_dictionary(circuits) 
    return (rho_hom_eval, a_tilde_update, b_tilde_update, circuit_dictionary, None, None, None, None)

In [ ]:
def no_qubits(circuit):
    """
    Compute the number of qubits based on the circuit.
    
    Args: 
        - circuit: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns: 
        - number_of_qubits: an integer corresponding to the number of qubits on which the circuit is acting upon.
    """
    number_of_qubits = 0
    first_layer = tcl_layers([circuit[0]])[0]
    for i in first_layer:
        if i == "CNOT":
            number_of_qubits = number_of_qubits + 2
        else:
            number_of_qubits = number_of_qubits + 1
    return(number_of_qubits)

In [ ]:
def flatten_circuit(circuits):
    """
    Extract all the gates applied to each qubit in the same ordering of the qubits.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns: 
        - flattened_circuit: a nested list, where each sub-list contains the gates applied to each qubit 
        and each layer. If the CNOT gate is present, then CNOT(1) indicates the control qubit
        and CNOT(2) indicates the target qubit. 
    """
    number_of_qubits = no_qubits(circuits)
    individual_gates = tcl_layers(circuits)
    number_of_layers = len(individual_gates)
    flattened_circuit = [[] for _ in range(number_of_qubits)]
    for i in range(number_of_layers):
        counter = 0
        for j in range(number_of_qubits):
            if counter + 1 > number_of_qubits:
                break
            
            if individual_gates[i][j] != "CNOT":
                flattened_circuit[counter].append(individual_gates[i][j])
                counter = counter + 1
            else:
                flattened_circuit[counter].append("CNOT(1)")
                flattened_circuit[counter+1].append("CNOT(2)")
                counter = counter + 2

    return flattened_circuit

In [ ]:
def number_of_tgates(circuit):
    """
    Count the number of T gates in a circuit.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the 
        list represents one layer of the circuit. 
        
    Returns: 
        - t_gate_count: total number of T gates in a circuit as an integer
    """
    L = tcl_layers(circuit)
    t_gate_count = 0
    for i in L:
        for j in i:
            if j == "T":
                t_gate_count = t_gate_count + 1
    return t_gate_count

In [ ]:
def wire_dictionary(circuits):
    """
    Construct a dictionary which contains information about the order of wires that act upon each qubit, 
    whether a T gate has been applied to a qubit during the evaluation of the EPR scheme or not, how many T gates
    will be acted upon each qubit and how many T gates have been applied during each iteration of the EPR evaluation 
    process. 
    This dictionary will provide a systematic way of keeping track of wires acting upon each qubit. 
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns:
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 
    """
    flat_circuit = flatten_circuit(circuits)
    number_of_qubits = len(flat_circuit)
    circuit_dictionary = {}
    for i in range(1, number_of_qubits+1):
        key = str(i)
        value = [i, False, number_of_tgates(flat_circuit[i-1]), 0]
        circuit_dictionary[key] = value
    return circuit_dictionary

In [ ]:
def no_wires(circuits):
    """
    Count the total of number of wires required to run a circuit using the EPR evaluation scheme.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns:
        - number_of_wires: total number of wires, which is calculated by 
            number of qubits in the input message + 2 * number of T gates
    """
    number_of_qubits = no_qubits(circuits)
    t_gate_count = number_of_tgates(circuits)
    number_of_wires = number_of_qubits + 2 * t_gate_count
    return number_of_wires

In [ ]:
def tgate_index(dictionary):
    """
    Construct a list consisting of the orderings of qubits on which a T gate is being applied.
    
    Args:
        - dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process.
            
    Returns: 
        - t_index_list: a list which records the orderings of qubits on which a T gate is being applied. 
    """
    t_index_list = []
    for key, value in dictionary.items():
        if dictionary[key][2] > 0:
            t_index_list.append(dictionary[key][0]-1)
    return t_index_list

In [ ]:
def iterated_tensor_prod(M, n): # Compute M^{\otimes n}
    """
    Compute the matrix M^{\otimes n} with the convention that if n = 0, then the output is the integer 1. 
    
    Args:
        - M: A matrix
        - n: a non-negative integer
        
    Returns:
        - iterated_tensored_m: M^{\otimes n}
    """
    if n == 0:
        return 1
    iterated_tensored_m = M
    for i in range(n-1):
        iterated_tensored_m = np.kron(iterated_tensored_m, M)
    return iterated_tensored_m

In [ ]:
def number_of_t_gates_applied(dictionary, qubit_order):
    """
    Count the total number of T gates applied to a given qubit during the evaluation procedure of the EPR scheme.
    
    Args: 
        - dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process.
        qubit_order: the order of a qubit in a n-qubit system, an integer between 1 to n. 
        
    Returns: 
        - number_of_t_gates: an integer counting the number of T gates applied during
    """
    number_of_t_gates = 0
    for key in dictionary:
        if int(key)-1 < qubit_order:
            number_of_t_gates += dictionary[key][2]  # Access the third element of the list
    return number_of_t_gates

In [ ]:
def controlled_not_constructor(control, target, number_of_qubits): 
    """
    This function applies the controlled-not operation on the target $t$ which is being controlled by $c$, 
    where $t < c$ or $c > t$ and the operation is being applied on a $n$-qubit system. 
    The operator matrix is defined as follows:

    if $t < c$: 

    $$CX_{c, t, n} = \left[ I^{\otimes(c-1)} \otimes | 0 \rangle \langle 0 | \otimes I^{\otimes(n-c)} \right] + 
    \left[ I^{\otimes(t-1)} \otimes X \otimes I^{\otimes(c-t-1)} \otimes | 1 \rangle \langle 1 | \otimes I^{\otimes (n-c)} \right]
    $$

    if $t > c$:

    $$
    CX_{c, t, n} = \left[ I^{\otimes(c-1)} \otimes | 0 \rangle \langle 0 | \otimes I^{\otimes(n-c)} \right]
    + \left[ I^{\otimes(c-1)} \otimes | 1 \rangle \langle 1 | \otimes I^{\otimes(t-c-1)} \otimes X \otimes I^{\otimes (n-t)}  \right]
    $$

    $I^{\otimes 0}$ is defined to be the real number $1$. 

    This function is used when applying the `T` gate during the EPR scheme.
    
    Args: 
        - control: the order of the control wire
        - target: the order of the target wire
        - number_of_qubits: number of qubits in the quantum system
        
    Returns:
        - controlled_not: the unitary operator which applies the X gate on 
        wire `target` if the `control` wire is 1.  
    """
    do_nothing_operator = np.kron(np.eye(2 ** (control - 1)), pure_to_density(zero_state()))
    do_nothing_operator = np.kron(do_nothing_operator, np.eye(2 ** (number_of_qubits - control)))
    if target < control: 
        flip_target_operator = np.eye(2 ** (target - 1))
        flip_target_operator = np.kron(flip_target_operator, X)
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (control - target - 1)))
        flip_target_operator = np.kron(flip_target_operator, pure_to_density(one_state()))
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (number_of_qubits - control)))
        
    else:
        flip_target_operator = np.eye(2 ** (control - 1))
        flip_target_operator = np.kron(flip_target_operator, pure_to_density(one_state()))
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (target - control - 1)))
        flip_target_operator = np.kron(flip_target_operator, X)
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (number_of_qubits - target)))

    controlled_not = do_nothing_operator + flip_target_operator
    return controlled_not

In [ ]:
def encrypt_one_bit(a):
    """
    Encrypt a bit `a` using the BFV classical fully homomorphic encryption scheme.
    
    Args:
        - a: a list containing a bit to be encrypted
    
    Returns:
        - a_enc: a list containing the ciphertext resulted from the encryption of the bit 
    """
    padded_a = pad_zeros(a, degree)
    encoded_a = [encoder.encode(i) for i in padded_a]
    a_enc = [encryptor.encrypt(i) for i in encoded_a]
    
    return a_enc

In [ ]:
def t_gate_sequence(circuit):
    """
    Construct a nested list where each nested list contains the information about the 
    order of the qubit on which a T gate is applied and the corresponding order of the layer
    as it appears in the circuit sequentially.
    
    Args:
        - circuit: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit.
    
    Returns:
        - t_gate_sequence_over_time: a nested list where each element is a list with the order of the qubit and the 
        order of the layer in which it appears in the circuit. 
    """
    individual_gates = tcl_layers(circuit)
    t_gate_count = number_of_tgates(circuit)
    t_gate_sequence_over_time = [[]]*t_gate_count
    t_gate_count_over_time = 0
    for gate_order in range(len(individual_gates)):
        qubit_order = 0
        for quasi_qubit_order in range(len(individual_gates[gate_order])):
            if individual_gates[gate_order][quasi_qubit_order] == "T":
                t_gate_sequence_over_time[t_gate_count_over_time] = [qubit_order, gate_order]
                t_gate_count_over_time = t_gate_count_over_time + 1
                
            if individual_gates[gate_order][quasi_qubit_order] in ("T","H","P","X","Z","I"):
                qubit_order = qubit_order + 1
            
            if individual_gates[gate_order][quasi_qubit_order] == "CNOT":
                qubit_order = qubit_order + 2
                
    return t_gate_sequence_over_time

In [ ]:
def sub_zero_ab(polynomial):
    """
    Substitute 0 for variables `a` and `b` in a symbolic polynomial which may consists `a` or `b`.
    
    Args:
        - polynomial: a symbolic polynomial which may consist `a` or `b`
        
    Returns:
        - eval_polynomial: the evaluated `polynomial` where 0 has been substituted for variables `a` and `b` 
    """
    string_polynomial = str(polynomial)
    eval_polynomial = polynomial
    if "a" in string_polynomial:
        for i in list(polynomial.free_symbols):
            if "a" in str(i):
                eval_polynomial = eval_polynomial.subs(i,0)
                
    if "b" in string_polynomial:
        for i in list(polynomial.free_symbols):
            if "b" in str(i):
                eval_polynomial = eval_polynomial.subs(i,0) 
                
    return eval_polynomial

In [ ]:
def get_subscript_superscript(monomial):
    """
    Extract d1 and d2 from a monomial of the form a_{d1}^{d2}. This information will be used 
    to determine the gate order and the qubit order in the evaluation of a polynomial.
    
    Args:
        - monomial: a symbolic expression of the form "c_{d1}^{d2}" or "k_{d1}^{d2}".
        
    Returns: 
        - subscript: the integer that is the subscript of the monomial. 
        - superscript: the integer that is the superscript of the monomial. 
    """
    # Define a regular expression pattern to match integers preceded by "_" and followed by "^"
    pattern = re.compile(r'_(\d+)\^(\d+)')
    matches = pattern.findall(monomial)
    
    # Extract subscript and superscript
    subscript = int(matches[0][0]) 
    superscript = int(matches[0][1])  
    
    return (subscript, superscript)

In [ ]:
def evaluate_expression(polynomial, k_measurements, c_measurements):
    """
    Evaluate a polynomial with variables in terms of `k` and `c` whose values are given 
    in the lists `k_measurements` and `c_measurements`.
    
    Args: 
        - polynomial: a symbolic polynomial consisting of variables `a`, `b`, `c` and `k`
        - k_measurements: a list of measured `k` values
        - c_measurements: a list of measured `c` values
        
    Returns: 
        - evaluated_polynomial: the evaluated value of the polynomial after substituting the values for `k` and `c`
        modulo 2
    """
    polynomial_no_ab = sub_zero_ab(polynomial)
    evaluated_polynomial = polynomial_no_ab
    for i in list(polynomial_no_ab.free_symbols):
        if "c" in str(i):
            c_gate_order, c_qubit_order = get_subscript_superscript(str(i))
            evaluated_polynomial = evaluated_polynomial.subs(i, c_measurements[c_qubit_order][c_gate_order])

        if "k" in str(i):
            k_gate_order, k_qubit_order = get_subscript_superscript(str(i))
            evaluated_polynomial = evaluated_polynomial.subs(i, k_measurements[k_qubit_order][k_gate_order])
    
    evaluated_polynomial = int(evaluated_polynomial) % 2
    return evaluated_polynomial

In [ ]:
def phase_gate_polynomial(circuits):
    """
    Construct the symbolic expression for evaluating the phase gate exponent before applying 
    a T gate, as well as the the symbolic expression of the key updates at the end of the 
    circuit evaluation. 
    
    The symbolic construction is defined in the following way depending on the input gate:
    (Table generated using the `tabulate` package)
    
    ╒═════════╤═════════════════════════╤═════════════════════════════════════╕
    │ Gate    │ key                     │ updated key                         │
    ╞═════════╪═════════════════════════╪═════════════════════════════════════╡
    │ I, X, Z │ (a,b)                   │ (a,b)                               │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ H       │ (a,b)                   │ (b,a)                               │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ P       │ (a,b)                   │ (a, a + b)                          │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ CNOT    │ ((a_1, b_1),(a_2, b_2)) │ ((a_1, b_1 + b_2),(a_1 + a_2), b_2) │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ T       │ (a,b)                   │ (a+c, a+(a*c)+b+k)                  │
    ╘═════════╧═════════════════════════╧═════════════════════════════════════╛
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit.
    
    Returns:
        - phase_gate_polynomials: a list containing the symbolic polynomial before the applicaton 
        of a T gate and all the key updates at the end of the circuit evaluation. 
    """
    individual_gates = tcl_layers(circuits)
    number_of_qubits = no_qubits(circuits)
    key_update_list = [[symbols(f'a_{1}^{i}'),symbols(f'b_{1}^{i}')] for i in range(number_of_qubits)]
    circuit_dictionary = wire_dictionary(circuits)
    t_index_list = tgate_index(circuit_dictionary)
    flat_circuit = flatten_circuit(circuits)
    phase_gate_polynomials = [[] for i in range(number_of_qubits)]
    
    for i in t_index_list:
        phase_gate_polynomials[i] = [key_update_list[i][0]]
    
    
    for gate_order in range(len(individual_gates)):
        
        # we create a seperate variable to keep track of the order of the qubit
        # to handle the case when a CNOT is applied after which
        # the order of qubit will increase by 2. 
        qubit_order = 0
        for quasi_qubit_order in range(number_of_qubits):
            if qubit_order < number_of_qubits:
                current_gate = individual_gates[gate_order][quasi_qubit_order]
                
                if current_gate in ("I","X","Z"):
                    key_update_list[qubit_order] = [key_update_list[qubit_order][0], key_update_list[qubit_order][1]]
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] > 0:
                        phase_gate_polynomials[qubit_order].append(key_update_list[qubit_order][0])
                    
                    elif gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]] 
                    
                    
                    qubit_order += 1

                if current_gate == "H":
                    key_update_list[qubit_order] = [key_update_list[qubit_order][1], key_update_list[qubit_order][0]]
                    
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] > 0:
                        phase_gate_polynomials[qubit_order].append(key_update_list[qubit_order][0])
                        
                    elif gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]]
                    
                    qubit_order += 1


                if current_gate == "P":
                    key_update_list[qubit_order] = [key_update_list[qubit_order][0], 
                                                      Add(key_update_list[qubit_order][0], 
                                                          key_update_list[qubit_order][1])]
                    
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] > 0:
                        phase_gate_polynomials[qubit_order].append(key_update_list[qubit_order][0])
                        
                    elif gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]]
            
            
                    qubit_order += 1


                if current_gate == "CNOT":
                    key_update_list[qubit_order] = [key_update_list[qubit_order][0],
                                                     Add(key_update_list[qubit_order][1],
                                                        key_update_list[qubit_order+1][1]
                                                        )
                                                     ]
                    key_update_list[qubit_order+1] = [Add(key_update_list[qubit_order+1][0],
                                                           key_update_list[qubit_order][0]),
                                                       key_update_list[qubit_order+1][1]]


                    if gate_order + 1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] > 0:
                        phase_gate_polynomials[qubit_order].append(key_update_list[qubit_order][0])
                        
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]]

                    if gate_order + 1 < len(individual_gates) and flat_circuit[qubit_order+1][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+2)][3] > 0:
                        phase_gate_polynomials[qubit_order+1].append(key_update_list[qubit_order+1][0])
                        
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order+1][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+2)][3] == 0:
                        phase_gate_polynomials[qubit_order+1] = [key_update_list[qubit_order+1][0]]
                        
                    if gate_order + 1 == len(individual_gates):
                        phase_gate_polynomials[qubit_order].append(
                            [key_update_list[qubit_order][0], key_update_list[qubit_order][1]]
                        )
                        phase_gate_polynomials[qubit_order+1].append(
                            [key_update_list[qubit_order+1][0], key_update_list[qubit_order+1][1]]
                        )
                        

                    qubit_order += 2

                if current_gate == "T":
                    no_t_gates_ith_wire = circuit_dictionary[str(qubit_order+1)][3]
                    c_symbol = symbols(f'c_{no_t_gates_ith_wire}^{qubit_order}')
                    k_symbol = symbols(f'k_{no_t_gates_ith_wire}^{qubit_order}')

                    key_update_list[qubit_order] = [key_update_list[qubit_order][0],
                                                     Add(Add(Add(key_update_list[qubit_order][0], 
                                                         Mul(key_update_list[qubit_order][0],c_symbol)), 
                                                         k_symbol), key_update_list[qubit_order][1])]
                    
                    if gate_order == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]]
                        circuit_dictionary[str(qubit_order+1)][3] = no_t_gates_ith_wire+1
                                                      
                    if gate_order > 0 and gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] == 0:
                        phase_gate_polynomials[qubit_order] = [key_update_list[qubit_order][0]]
                                                     
                        circuit_dictionary[str(qubit_order+1)][3] = no_t_gates_ith_wire+1
                    
                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] == "T" and circuit_dictionary[str(qubit_order+1)][3] > 0:
                        phase_gate_polynomials[qubit_order].append(key_update_list[qubit_order][0])
                        circuit_dictionary[str(qubit_order+1)][3] = no_t_gates_ith_wire+1

                    if gate_order+1 < len(individual_gates) and flat_circuit[qubit_order][gate_order+1] != "T":
                        circuit_dictionary[str(qubit_order+1)][3] = no_t_gates_ith_wire+1
                    qubit_order += 1
                    
                if gate_order+1 == len(individual_gates) and current_gate != "CNOT":
                    phase_gate_polynomials[qubit_order-1].append(
                        [key_update_list[qubit_order-1][0], key_update_list[qubit_order-1][1]]
                    )
             
    return phase_gate_polynomials

In [ ]:
def epr_eval(circuits, enc_psi, a_tilde, b_tilde):    
    """
    Perform a circuit decomposed into Clifford + T gates on the encrypted quantum state
    homomorphically and update the keys. For every T gate, store the updated value of "a_tilde" before the application 
    of the T gate. Upon applying the T gate, make a measurement, encrypt the measured bit using the 
    public key component of the classical homomorphic encryption scheme and store the ciphertext in a list, which will 
    be returned at the end of the module. 
    For more information, refer to the EPR scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        - enc_psi: a vector representation of the encrypted pure state |psi>_{enc}
        - a_tilde: a list of classical ciphertexts which contain the information about the X-Pauli gate exponents 
        - b_tilde: a list of classical ciphertexts which contain the information about the Z-Pauli gate exponents 

    Returns: 
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - a_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the X-Pauli gate exponents 
        - b_tilde_update: a list of updated classical ciphertexts 
        which contain the information about the Z-Pauli gate exponents 
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process.
        
        If there are any T gates in the circuit:
            - t_gate_sequence_over_time: a nested list where each element is a list with the order of the qubit and the 
        order of the layer in which it appears in the circuit. 
            - c_encrypted: a list of ciphertexts resulting from the measurement during the evaluation of the EPR scheme
            - p_exponents: a list of ciphertexts recording the updated "a_tilde" value before the application of a
            T gate.
    """
    # Define variables for bookkeeping
    flat_circuit = flatten_circuit(circuits)
    acting_wire_dictionary = wire_dictionary(circuits) 
    number_of_qubits = no_qubits(circuits)
    t_gate_count = number_of_tgates(circuits)
    number_of_wires = no_wires(circuits)
    individual_gates = tcl_layers(circuits)
    number_of_layers = len(individual_gates)
    t_index_list = tgate_index(acting_wire_dictionary)
    
    # initialize list
    to_be_encrypted = [0] * (number_of_layers + 1)
    c_encrypted = [[] for i in range(number_of_qubits)] # encrypted to-be-measured values of c 
                                                        # will be stored in this list

    
    # if there are no T gates in the circuit, then apply the Clifford scheme evaluation procedure
    if t_gate_count == 0:
        return cl_eval(circuits, enc_psi, a_tilde, b_tilde)
    
    
    elif t_gate_count > 0:
        p_exponents = [[] for i in range(number_of_qubits)] # phase exponents will be stored in this list 
                                                            # which then will be used for decryption
        
        # initialize the first phase gate exponent according to the a_tilde values for any
        # qubit on which a T gate is going to be applied
        for i in t_index_list:
            p_exponents[i] = [a_tilde[i]]
            
        # append EPR pairs to the system
        enc_psi_eval = np.kron(enc_psi, iterated_tensor_prod(gen_epr(), t_gate_count))
        
        # define lists for storing ciphertexts and key updates
        a_tilde_intermediate_update = [copy.deepcopy(to_be_encrypted) for i in range(number_of_qubits)]
        b_tilde_intermediate_update = [copy.deepcopy(to_be_encrypted) for i in range(number_of_qubits)]
        
        for i in range(number_of_qubits):
            a_tilde_intermediate_update[i][0] = a_tilde[i]
            b_tilde_intermediate_update[i][0] = b_tilde[i]
    
    # density matrix of the encrypted state |psi>_{enc}
    rho_hom_eval = pure_to_density(enc_psi_eval)
    
    # apply SWAP gates
    for qubit_order in range(number_of_qubits):
        if acting_wire_dictionary[str(qubit_order+1)][2] == 0:
            continue 
        else: 
            # number of wires before the first T gate on the i^ht qubit
            n_w_b_t_f_t = number_of_qubits + number_of_t_gates_applied(acting_wire_dictionary, qubit_order) * 2 
            
            # number of T gates on the i^th qubit
            n_t_q_n = acting_wire_dictionary[str(qubit_order+1)][2] 
            
            starting_wire = n_w_b_t_f_t + 1
            current_t_wire = starting_wire
            for actingTwire in range(n_t_q_n * 2):
                current_t_wire = starting_wire + actingTwire
                for remaining_qubits in range(number_of_qubits-acting_wire_dictionary[str(qubit_order+1)][0]):
                    current_circuit = iterated_tensor_prod(I, current_t_wire-2)
                    current_circuit = np.kron(current_circuit, SWAP)
                    current_circuit = np.kron(current_circuit, 
                                              iterated_tensor_prod(I,number_of_wires - current_t_wire))
                    rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
                    current_t_wire = current_t_wire - 1
                    
    # evaluation procedure begins
    for gate_order in range(len(individual_gates)):
        
        # we create a seperate variable to keep track of the order of the qubit
        # to handle the case when a CNOT is applied after which
        # the order of qubit will increase by 2. 
        qubit_order = 0
        
        for quasi_qubit_order in range(len(individual_gates[gate_order])):
            if acting_wire_dictionary[str(qubit_order+1)][3] == 0:
                acting_wire = acting_wire_dictionary[str(qubit_order+1)][0] + (
                    2*number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
                )
            else: 
                acting_wire = acting_wire_dictionary[str(qubit_order+1)][0]
            
            current_gate = individual_gates[gate_order][quasi_qubit_order]
            
            # Clifford Gate
            if current_gate != "T":   
                if current_gate in ["I", "X", "Z"]:
                    a_tilde_intermediate_update[qubit_order][gate_order+1], b_tilde_intermediate_update[qubit_order][gate_order+1] = clifford_key_updates("I", a_tilde_intermediate_update[qubit_order][gate_order], 
                                           b_tilde_intermediate_update[qubit_order][gate_order])

                    if current_gate == "I":
                        current_circuit = iterated_tensor_prod(I, number_of_wires)


                    elif current_gate == "X":
                        current_circuit = iterated_tensor_prod(I, acting_wire-1)
                        current_circuit = np.kron(current_circuit, X)
                        current_circuit = np.kron(current_circuit, 
                                                  iterated_tensor_prod(I,number_of_wires - acting_wire))

                    else: 
                        current_circuit = iterated_tensor_prod(I, acting_wire-1)
                        current_circuit = np.kron(current_circuit, Z)
                        current_circuit = np.kron(current_circuit, 
                                                  iterated_tensor_prod(I,number_of_wires - acting_wire))

                    qubit_order += 1

                elif current_gate == "P":
                    current_circuit = iterated_tensor_prod(I, acting_wire-1)
                    current_circuit = np.kron(current_circuit, P)
                    current_circuit = np.kron(current_circuit, 
                                              iterated_tensor_prod(I,number_of_wires - acting_wire))

                    a_tilde_intermediate_update[qubit_order][gate_order+1], b_tilde_intermediate_update[qubit_order][gate_order+1] = clifford_key_updates("P", a_tilde_intermediate_update[qubit_order][gate_order], 
                                           b_tilde_intermediate_update[qubit_order][gate_order])


                    qubit_order += 1
                    

                elif current_gate == 'H':
                    current_circuit = iterated_tensor_prod(I, acting_wire-1)
                    current_circuit = np.kron(current_circuit, H)
                    current_circuit = np.kron(current_circuit, iterated_tensor_prod(I,number_of_wires - acting_wire))


                    a_tilde_intermediate_update[qubit_order][gate_order+1], b_tilde_intermediate_update[qubit_order][gate_order+1] = clifford_key_updates("H", a_tilde_intermediate_update[qubit_order][gate_order], 
                                           b_tilde_intermediate_update[qubit_order][gate_order])
                    
                    qubit_order += 1
                
                # CNOT
                else:
                    control_wire = acting_wire
                    if acting_wire_dictionary[str(qubit_order+2)][3] == 0:
                        target_wire = acting_wire_dictionary[str(qubit_order+2)][0] + (
                            2*number_of_t_gates_applied(acting_wire_dictionary, qubit_order+1)
                        )
                    else: 
                        target_wire = acting_wire_dictionary[str(qubit_order+2)][0]
                        
                    current_circuit = controlled_not_constructor(control_wire, target_wire, number_of_wires)


                    temp1, temp2 = clifford_key_updates(
                        "CNOT", [a_tilde_intermediate_update[qubit_order][gate_order], a_tilde_intermediate_update[qubit_order+1][gate_order]], 
                        [b_tilde_intermediate_update[qubit_order][gate_order], b_tilde_intermediate_update[qubit_order+1][gate_order]]
                    )

                    a_tilde_intermediate_update[qubit_order][gate_order+1] = temp1[0]
                    a_tilde_intermediate_update[qubit_order+1][gate_order+1] = temp1[1]
                    b_tilde_intermediate_update[qubit_order][gate_order+1] = temp2[0]
                    b_tilde_intermediate_update[qubit_order+1][gate_order+1] = temp2[1]

                    qubit_order += 2
                    
                rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
    
            elif current_gate == "T":
                # The following two 'if' statements store the `a_tilde_intermediate_update` ciphertext
                # before applying the T gate:
                # - if a T gate has not been yet applied to this qubit, then create a fresh list
                # - if a T gate has been applied at this point of the evaluation then append to the list
                if acting_wire_dictionary[str(qubit_order+1)][3] == 0:
                    p_exponents[qubit_order] = [a_tilde_intermediate_update[qubit_order][gate_order]]

                if acting_wire_dictionary[str(qubit_order+1)][3] > 0:
                    p_exponents[qubit_order].append(a_tilde_intermediate_update[qubit_order][gate_order])

                # keeping track of the control wire and target wire
                if acting_wire_dictionary[str(qubit_order+1)][1] == True: 
                    target_wire = acting_wire
                    control_wire = acting_wire + 2

                if acting_wire_dictionary[str(qubit_order+1)][1] == False:
                    acting_wire_dictionary[str(qubit_order+1)][1] = True
                    no_tgates_before_ith_qubit = number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
                    target_wire = acting_wire
                    control_wire = (qubit_order) + 2*no_tgates_before_ith_qubit + 2

                
                # apply the T gate
                t_gate_operator = np.kron(iterated_tensor_prod(I, target_wire-1), T)
                t_on_rho_hom_eval = qgate_on_density(np.kron(
                t_gate_operator,iterated_tensor_prod(I, number_of_wires - target_wire)), rho_hom_eval)

                # apply the CNOT gate
                cnot_on_t_rho_hom_eval = qgate_on_density(controlled_not_constructor(control_wire, target_wire, number_of_wires), t_on_rho_hom_eval)
                
                # make measurement and obtain the post-measurement state and the classical bit c
                pms, c = post_measurement_state(cnot_on_t_rho_hom_eval, [target_wire-1], 
                                          measurement(partial_trace(cnot_on_t_rho_hom_eval, [target_wire-1])))

                # encrypt bit c using the public key
                c_tilde = encrypt_one_bit([int(c)])
                
                # append the ciphertext to a list storing the encryption of measured 'c' values
                c_encrypted[qubit_order].append(c_tilde[0])
                
                # update `a_tilde` value homomorphically
                a_tilde_add_c = evaluator.add(a_tilde_intermediate_update[qubit_order][gate_order], c_tilde[0])
                
                # update `b_tilde` value homomorphically
                a_tilde_times_c_tilde = evaluator.multiply(a_tilde_intermediate_update[qubit_order][gate_order], c_tilde[0], relin_key)
                a_tilde_times_c_tilde_add_a_tilde = evaluator.add(a_tilde_times_c_tilde, a_tilde_intermediate_update[qubit_order][gate_order])
                a_tilde_times_c_tilde_add_a_tilde_add_b_tilde = evaluator.add(
                    b_tilde_intermediate_update[qubit_order][gate_order], a_tilde_times_c_tilde_add_a_tilde)

                # update the lists
                a_tilde_intermediate_update[qubit_order][gate_order+1] = a_tilde_add_c
                b_tilde_intermediate_update[qubit_order][gate_order+1] = a_tilde_times_c_tilde_add_a_tilde_add_b_tilde  

                # perform the necessary updates
                acting_wire_dictionary[str(qubit_order+1)][0] = control_wire 
                acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
                rho_hom_eval = pms
                qubit_order += 1
     
    # retain the last key updates
    a_tilde_update = [a_tilde_intermediate_update[i][-1] for i in range(number_of_qubits)]
    b_tilde_update = [b_tilde_intermediate_update[i][-1] for i in range(number_of_qubits)]
    
    # fresh copy of the circuit dictionary required for decryption
    circuit_dictionary = wire_dictionary(circuits) 
    
    t_gate_sequence_over_time = t_gate_sequence(circuits)
    key_polynomials = phase_gate_polynomial(circuits)
    return (rho_hom_eval, a_tilde_update, b_tilde_update, circuit_dictionary, key_polynomials, t_gate_sequence_over_time, c_encrypted, p_exponents)